# Проект: Исследование надежности заемщиков


## Данные


В наличии были следующие данные:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Задача

На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.


## Используемые библиотеки

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

Создание столбца `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создание столбца `purpose_category`с категориями:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

In [22]:
data_from_children = pd.DataFrame()
data_from_children['count_of_debtors'] = data.groupby('children')['debt'].count()# Подсчет количества семей с определенным количеством детей, взявших кредит
data_from_children['sum_of_debtors'] = data.groupby('children')['debt'].sum()# Подсчет количества семей с определенным количеством детей, имеющих задолженность по кредиту 
data_from_children['percentage_of_debtors'] = data_from_children['sum_of_debtors']/data_from_children['count_of_debtors']*100# Расчет процента семей с определенным количеством детей, имеющих задолженность
print(data_from_children)


          count_of_debtors  sum_of_debtors  percentage_of_debtors
children                                                         
0                    14107            1063               7.535266
1                     4809             444               9.232689
2                     2052             194               9.454191
3                      330              27               8.181818
4                       41               4               9.756098
5                        9               0               0.000000


**Вывод:** Результаты для семей с 4 и 5 детьми не учитываются при составлении вывода, так как являются статистически не значимыми. Можно сказать, что при отсутствии детей, вероятность выплатить кредит в срок выше, чем с 1 ребенком или 2 детьми в семье. При этом разница в проценте должников между семьями с 1 ребенком и 2 детьми мала. **Следовательно, есть зависимость между наличием детей и возвратом кредита в срок.** 

In [23]:
data_family_status= pd.DataFrame()
data_family_status['count_of_debtors'] = data.groupby('family_status')['debt'].count()# Подсчет количества клиентов с определенным семейным статусом, взявших вредит
data_family_status['sum_of_debtors'] = data.groupby('family_status')['debt'].sum()# Подсчет количества клиентов с определенным семейным статусом, имеющих задолженность по кредиту 
data_family_status['percentage_of_debtors'] = data_family_status['sum_of_debtors']/data_family_status['count_of_debtors']*100# Расчет процента клиентов с определенным семейным статусом, имеющих задолженность
print(data_family_status.sort_values(by='percentage_of_debtors'))
# Ваш код будет здесь. Вы можете создавать новые ячейки.

                       count_of_debtors  sum_of_debtors  percentage_of_debtors
family_status                                                                 
вдовец / вдова                      951              63               6.624606
в разводе                          1189              84               7.064760
женат / замужем                   12266             927               7.557476
гражданский брак                   4146             385               9.286059
Не женат / не замужем              2796             273               9.763948


**Вывод:** результаты для пользователей с семейным статусом вдовец/вдова и в разводе можно отбросить как статистически не значимые. 
**Зависимость между семейным положением и возвратом кредита в срок есть.** Женатые/замужние заемщики вероятнее возвращают кредит в срок, чем не женатые/не замужние, что может быть связано с общим семейным бюждетом, характерным для брака.

In [24]:

data_from_income = pd.DataFrame()
#data['total_income_category_my'] = pd.qcut(data['total_income'], 4)# Разбивка данных на 4 одинаковые (по количеству клиентов) группы по уровню дохода
#data_from_income['count_of_debtors'] = data.groupby('total_income_category_my')['debt'].count()# Подсчет количества клиентов с определенным доходом, взявших вредит
#data_from_income['sum_of_debtors'] = data.groupby('total_income_category_my')['debt'].sum()# Подсчет количества клиентов с определенным доходом, имеющих задолженность по кредиту 
#data_from_income['percentage_of_debtors'] = data_from_income['sum_of_debtors']/data_from_income['count_of_debtors']*100# Расчет процента клиентов с определенным доходом, имеющих задолженность

data_from_income['count_of_debtors'] = data.groupby('total_income_category')['debt'].count()# Подсчет количества клиентов с определенным доходом, взявших вредит
data_from_income['sum_of_debtors'] = data.groupby('total_income_category')['debt'].sum()# Подсчет количества клиентов с определенным доходом, имеющих задолженность по кредиту 
data_from_income['percentage_of_debtors'] = data_from_income['sum_of_debtors']/data_from_income['count_of_debtors']*100# Расчет процента клиентов с определенным доходом, имеющих задолженность
    
display(data_from_income[data_from_income['count_of_debtors'] > 5000].sort_values(by='percentage_of_debtors'))

# Ваш код будет здесь. Вы можете создавать новые ячейки.

,count_of_debtors,sum_of_debtors,percentage_of_debtors
total_income_category,,,
B,5014,354,7.060231
C,15938,1353,8.489145


**Вывод:** Наименьшее количество должников характерно для группы людей с доходом от 195 тыс., для остальных групп должников разница мала.
**Зависимость между уровнем дохода и возвратом кредита в срок есть, при доходе от 195 тыс. рублей процент задолженности снижается.**

<div class="alert-info">
 С увеличением дохода процент должников снижается.</div>

In [25]:
data_from_purpose = pd.DataFrame()
data_from_purpose['count_of_debtors'] = data.groupby('purpose_category')['debt'].count()# Подсчет количества клиентов с определенной целью, взявших вредит
data_from_purpose['sum_of_debtors'] = data.groupby('purpose_category')['debt'].sum()# Подсчет количества клиентов с определенной целью, имеющих задолженность по кредиту 
data_from_purpose['percentage_of_debtors'] = data_from_purpose['sum_of_debtors']/data_from_purpose['count_of_debtors']*100# Расчет процента клиентов с определенной целью, имеющих задолженность

print(data_from_purpose.sort_values(by='percentage_of_debtors'))

# Ваш код будет здесь. Вы можете создавать новые ячейки.

                          count_of_debtors  sum_of_debtors  \
purpose_category                                             
операции с недвижимостью             10754             780   
проведение свадьбы                    2324             183   
получение образования                 3989             369   
операции с автомобилем                4281             400   

                          percentage_of_debtors  
purpose_category                                 
операции с недвижимостью               7.253115  
проведение свадьбы                     7.874355  
получение образования                  9.250439  
операции с автомобилем                 9.343611  


**Вывод:** Некорректно сравнивать кредит на недвижимость с другими целями из-за огромного различия в объеме данных по этим группам. 
Процент заемщиков с целью "свадьба" меньше, чем заемщиков с целью "образование" или "автомобиль", что может быть связано с суммой необходимой для данной цели. Чем меньше сумма кредита, тем меньше должников.


## Общий вывод.

**Задача проекта:** Влияет ли семейное положение и количество детей на факт погашения кредита в срок?
**Ответ:** Да, влияет. При оценке кредитоспособности клиента следует 
обратить внимание на семейное положение и наличие детей,
так как семейные клиенты и клиенты без детей
вероятнее выплатят кредит в срок.